<a href="https://colab.research.google.com/github/KeeJin/QuickDrawDoodleClassifier/blob/master/Quickdraw_Doodle_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickdraw Challenge with custom resnet-34 model from scratch
Number of categories: 20  
Done by: Cheo Kee Jin

## Import dependencies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import os
from glob import glob
import ntpath

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Add, MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.models import Model

## Use TPU

In [ ]:
try:
    device_name = os.environ['COLAB_TPU_ADDR']
    TPU_ADDRESS = 'grpc://' + device_name
    print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
    print('TPU not found')

Found TPU at: grpc://10.6.110.82:8470


In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    
# Select appropriate distribution strategy
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)

print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.6.110.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.6.110.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.6.110.82:8470']
Number of accelerators:  8


## Download dataset and preprocess data

In [ ]:
!wget -qq https://www.dropbox.com/s/gdlb8dnjzcly51o/quickdraw.zip
  
!unzip -qq quickdraw.zip

!rm -r __MACOSX
!rm quickdraw.zip

!ls

quickdraw  sample_data


In [ ]:
!ls quickdraw

'alarm clock.npy'   bee.npy		 cookie.npy	  lollipop.npy
 ambulance.npy	   'birthday cake.npy'	 donut.npy	 'palm tree.npy'
 angel.npy	    brain.npy		 eyeglasses.npy   pig.npy
 ant.npy	    cactus.npy		 face.npy	  postcard.npy
 banana.npy	    cat.npy		 fish.npy	  rainbow.npy


In [ ]:
arr = np.load('./quickdraw/bee.npy')
arr.shape

(120890, 784)

In [ ]:
file_names = glob('./quickdraw/*.npy')

In [ ]:
file_names

['./quickdraw/alarm clock.npy',
 './quickdraw/cactus.npy',
 './quickdraw/ant.npy',
 './quickdraw/pig.npy',
 './quickdraw/cookie.npy',
 './quickdraw/ambulance.npy',
 './quickdraw/angel.npy',
 './quickdraw/eyeglasses.npy',
 './quickdraw/cat.npy',
 './quickdraw/postcard.npy',
 './quickdraw/brain.npy',
 './quickdraw/face.npy',
 './quickdraw/banana.npy',
 './quickdraw/birthday cake.npy',
 './quickdraw/fish.npy',
 './quickdraw/bee.npy',
 './quickdraw/rainbow.npy',
 './quickdraw/palm tree.npy',
 './quickdraw/donut.npy',
 './quickdraw/lollipop.npy']

In [ ]:
# make some class names
class_names = []

for file in file_names:
  name = ntpath.basename(file)
  class_names.append(name[:-4])
  
  
print(class_names)

['alarm clock', 'cactus', 'ant', 'pig', 'cookie', 'ambulance', 'angel', 'eyeglasses', 'cat', 'postcard', 'brain', 'face', 'banana', 'birthday cake', 'fish', 'bee', 'rainbow', 'palm tree', 'donut', 'lollipop']


In [ ]:
arr = np.load(file_names[0])
arr.shape

(123399, 784)

In [ ]:
x_data =np.array([])
y_labels =np.array([])

for i, filename in enumerate(file_names):
  labels = [i for j in range(35000)]
  arr = np.load(filename)
  arr = arr[:35000]
  if len(x_data) == 0:
    x_data = arr
    y_labels = np.asarray(labels)
  else:
    x_data = np.concatenate((x_data, arr))
    y_labels = np.concatenate((y_labels, labels))

In [ ]:
x_data.shape, y_labels.shape

((700000, 784), (700000,))

In [ ]:
# Shuffle
x_data, y_labels = shuffle(x_data, y_labels, random_state=42)

# Train test split
split = 0.9
x_train, x_test, y_train, y_test = train_test_split(x_data, y_labels, test_size = (1-split), random_state = 42)
x_train.shape, y_train.shape

((630000, 784), (630000,))

In [ ]:
# Example of an image array
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   5,
        37,  72,  68,  34,   4,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        83, 251, 255, 255, 255, 255, 251, 221, 188, 154, 120,  8

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Check that the set distribution is random
y_labels[:50]

(630000, 784) (630000,)
(70000, 784) (70000,)


array([18, 18,  9,  6,  2,  9, 19, 11,  1,  6,  5,  4, 16, 19,  0,  2,  5,
        5,  0, 18, 11, 10, 18, 12, 13, 14, 16,  3, 19,  6, 12,  5, 11, 19,
        6,  9,  7, 12,  8, 17, 18, 15, 16, 14, 10, 10, 16,  5,  2,  2])

## Prepare data for network

In [ ]:
img_rows, img_cols = 28, 28

x_train = (x_train.reshape(x_train.shape[0], img_rows, img_cols).astype('float32')) / 255
x_test = (x_test.reshape(x_test.shape[0], img_rows, img_cols).astype('float32')) / 255

y_train = tf.keras.utils.to_categorical(y_train, len(class_names))
y_test = tf.keras.utils.to_categorical(y_test, len(class_names))

In [ ]:
x_train.shape

(630000, 28, 28)

## Data Pipeline

In [ ]:
batch_size = 64*8 # Because we now have 8 TPU cores

train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_set.shuffle(630000)
train_set.repeat()
train_set = train_set.cache()
train_set = train_set.batch(batch_size, drop_remainder = True)

In [ ]:
valid_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))
valid_set.repeat()
train_set = train_set.cache()
valid_set = valid_set.batch(batch_size, drop_remainder = True)

## Model - hand-made resnet34

In [ ]:
def ConvBatchNorm(x, filters, kernel_size, strides):
  x = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = "same")(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  return x


def IdentityBlk(x_og, num_filters):
  x = ConvBatchNorm(x_og, num_filters, kernel_size = (1, 1), strides = 1)
  x = ConvBatchNorm(x, num_filters, kernel_size = (3, 3), strides = 1)
  x = Conv2D(filters = 4 * num_filters, kernel_size = (1, 1), strides = 1)(x)
  x = BatchNormalization()(x)
  x = Add()([x, x_og])
  x = ReLU()(x)
  return x


def projection_block(tensor, num_filters, strides):
    # left stream
    # minor change: reduction in the second conv block instead of the first --> feature of resnet V2
    x = ConvBatchNorm(tensor, filters = num_filters, kernel_size = 1, strides = 1)
    x = ConvBatchNorm(x, filters = num_filters, kernel_size = 3, strides = strides)
    x = Conv2D(filters = 4 * num_filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    
    # right stream
    proj_x = Conv2D(filters = 4 * num_filters, kernel_size = 1, strides = strides)(tensor)
    proj_x = BatchNormalization()(proj_x)
    x = Add()([x, proj_x])
    x = ReLU()(x)
    return x


def resnet_block(x, filters, reps, strides):
    x = projection_block(x, num_filters=filters, strides=strides)
    for _ in range(reps-1):
        x = IdentityBlk(x, num_filters=filters)
    return x

In [ ]:
with strategy.scope():
  model_input = Input(shape = (28, 28, 1), name = "input")
  x = ConvBatchNorm(model_input, filters = 64, kernel_size = 7, strides = 2)
  x = MaxPool2D(pool_size = 3, strides = 2, padding = "same")(x)
  x = resnet_block(x, filters = 64, reps = 3, strides = 1)
  x = resnet_block(x, filters = 128, reps = 4, strides = 2)
  x = resnet_block(x, filters = 256, reps = 6, strides = 2)
  x = resnet_block(x, filters = 512, reps = 3, strides = 2)
  x = GlobalAvgPool2D()(x)
  model_output = Dense(20, activation = "softmax")(x)

  model = Model(inputs = model_input, outputs = model_output, name = "resnet34")

In [ ]:
model.summary()

Model: "resnet34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 64)   3200        input[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 14, 14, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 14, 14, 64)   0           batch_normalization[0][0]        
___________________________________________________________________________________________

## Train

In [ ]:
reduce_LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor = 0.9, patience = 3, cooldown = 1, min_lr = 0.00001)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
%cd ../gdrive/My\ Drive/Colab\ Notebooks

/gdrive/My Drive/Colab Notebooks


In [ ]:
!mkdir savedweights

In [ ]:
# Callbacks
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = './savedweights/best_weightss.hdf5', 
             save_best_only = True, 
             monitor = "val_loss", 
             verbose = 1),
             reduce_LR,
             tf.keras.callbacks.EarlyStopping(monitor = "val_loss",
                                              min_delta = 0.005,
                                              patience = 7,
                                              verbose = 1)
             ]

In [ ]:
print(len(x_train))
print(len(x_test))

630000
70000


In [ ]:
lr = 0.008
train_steps = int(len(x_train)/batch_size)
val_steps = int(len(x_test)/batch_size)
epochs = 50

print(train_steps, val_steps)

with strategy.scope():
  opt = tf.keras.optimizers.Adam(lr = lr)
  model.compile(optimizer = opt, loss='categorical_crossentropy',metrics=['accuracy'])

1230 136


In [ ]:
history = model.fit(train_set, 
                    steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_set,
                   validation_steps = val_steps,
                    callbacks = callbacks)

Epoch 1/50
1229/1230 [============================>.] - ETA: 0s - loss: 1.1388 - accuracy: 0.7194
Epoch 00001: val_loss improved from inf to 0.57242, saving model to ./savedweights/best_weightss.hdf5
1230/1230 [==============================] - 73s 59ms/step - loss: 1.1382 - accuracy: 0.7196 - val_loss: 0.5724 - val_accuracy: 0.8303 - lr: 0.0080
Epoch 2/50
1230/1230 [==============================] - ETA: 0s - loss: 0.3930 - accuracy: 0.8864
Epoch 00002: val_loss improved from 0.57242 to 0.36491, saving model to ./savedweights/best_weightss.hdf5
1230/1230 [==============================] - 70s 57ms/step - loss: 0.3930 - accuracy: 0.8864 - val_loss: 0.3649 - val_accuracy: 0.8956 - lr: 0.0080
Epoch 3/50
1230/1230 [==============================] - ETA: 0s - loss: 0.3657 - accuracy: 0.8951
Epoch 00003: val_loss did not improve from 0.36491
1230/1230 [==============================] - 64s 52ms/step - loss: 0.3657 - accuracy: 0.8951 - val_loss: 0.6937 - val_accuracy: 0.8702 - lr: 0.0080
Epo

## Evaluate 

In [ ]:
!ls savedweights

best_weightss.hdf5


In [ ]:
model.load_weights("savedweights/best_weightss.hdf5")

In [ ]:
model.evaluate(valid_set, steps=val_steps)

136/136 [==============================] - 3s 19ms/step - loss: 0.2909 - accuracy: 0.9199


[0.2908700704574585, 0.9199362397193909]